In [1]:
import json
import csv
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
%%time
#load the MC38 chromosomes vcf file with snpEff annotation

with open('/home/raymond/Desktop/BlueSphere_Bio_Inc/MC38/MC38_BAM_somatic_mutation_calling/MC38_vcf_snpeff/MC38_chromosomes_snpEff.vcf') as f:
    vcf_reader = []
    MC38_chromosome_filter_dict = {}
    
    for i in f:
        if i.startswith('#'):
            continue
        else:
            i = i.split('\t')
            #print(i)
            vcf_reader.append(i)

#print(len(vcf_reader))
#print(vcf_reader[0])
        
    for i in vcf_reader:
        for j in i[7].split(';'):
            if j.startswith('DP'):
                DP = j.split('=')[-1]
        
            if j.startswith('MBQ'):
                MBQ = max(j.split('=')[-1].split(','))
            
            if j.startswith('MMQ'):
                MMQ = max(j.split('=')[-1].split(','))
        
            if j.startswith('TLOD'):
                TLOD = max(j.split('=')[-1].split(','))

        if float(TLOD) > 20 and float(MMQ) >= 40 and float(DP) > 30 and float(MBQ) > 30: 
            MC38_chromosome_filter_dict[i[0] + ':' + i[1]] = i 
        
print(len(MC38_chromosome_filter_dict))
print(list(MC38_chromosome_filter_dict.keys())[0], MC38_chromosome_filter_dict[list(MC38_chromosome_filter_dict.keys())[0]])
print(list(MC38_chromosome_filter_dict.keys())[-1], MC38_chromosome_filter_dict[list(MC38_chromosome_filter_dict.keys())[-1]])


#check chromosome

chrom_check = {}

for i in MC38_chromosome_filter_dict.keys():
    if i.split(':')[0] in chrom_check.keys():
        chrom_check[i.split(':')[0]] += 1
    else:
        chrom_check[i.split(':')[0]]  = 0
        
print(chrom_check)
print(sum(list(chrom_check.values())))

257247
chr1:3000441 ['chr1', '3000441', '.', 'T', 'G', '.', '.', 'AC=1;AN=2;ANN=G|intergenic_region|MODIFIER|CHR_START-Xkr4|CHR_START-Xkr4|intergenic_region|CHR_START-Xkr4|||n.3000441T>G||||||;AS_SB_TABLE=0,0|23,13;DP=36;ECNT=1;MBQ=0,37;MFRL=0,303;MMQ=60,60;MPOS=35;POPAF=7.30;SF=0;TLOD=131.33', 'GT:AD:F1R2:DP:F2R1:SB:AF', '0/1:0,36:0,22:36:0,14:0,0,23,13:0.970', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.\n']
chr19:61304054 ['chr19', '61304054', '.', 'T', 'C', '.', '.', 'AC=1;AN=2;ANN=C|intergenic_region|MODIFIER|Csf2ra-CHR_END|Csf2ra-CHR_END|intergenic_region|Csf2ra-CHR_END|||n.61304054T>C||||||;AS_SB_TABLE=22,17|3,9;DP=51;ECNT=6;MBQ=37,37;MFRL=278,378;MMQ=60,36;MPOS=43;POPAF=7.30;SF=18;TLOD=31.75', 'GT:AD:F1R2:DP:F2R1:SB:AF', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '0/1:39,12:21,7:51:18,5:22,17,3,9:0.255\n']
{'chr1': 20099, 'chr2': 19586, 'chr3': 15877, 'chr4': 25701, 'chr5': 21963, 'chr6': 

In [7]:
%%time
# reads cds region from ncbiRefSeq file 

def read_gtfcds(input_gtf):
    storage = []
    transId = ""
    with open(input_gtf) as gtf:
        temp = []
        valid = True
        for info in gtf:
            info = info.split()
            chromosome = info[0]
#           if chromosome == "chr10":
            tp = info[2]
            start = int(info[3])
            end = int(info[4])
            rev = info[6]
            transcriptId = info[11][1:len(info[11]) - 2]      
            geneId = info[9].split(';')[0].split('"')[1]

            #start of new transcript; cleanse global data
            if transId != transcriptId:
                transId = transcriptId
                temp = []
                valid = True

            #all with the same transcript id
            if valid:
                if rev == "+":
                    if tp == "CDS":
                        # creates a dictionary to improve performance
                        data = {}
                        data["chromosome"] = chromosome
                        data["transcriptId"] = transcriptId
                        data["geneId"] = geneId
                        data["start"] = start
                        data["end"] = end
                        data["rev"] = rev
                        temp.append(data)
                    #check if start of first codon is the same as the start codon
                    elif tp == "start_codon":
                        if len(temp) < 1 or start != temp[0]["start"]:
                            valid = False
                    #check if start of last CDS + 1 is the same as the start of the stop codon
                    elif tp == "stop_codon":
                        if len(temp) < 1 or start != temp[-1]["end"] + 1:
                            valid = False
                        else:
                            storage += temp
   
        return storage

input_mm10_gtf = "/home/raymond/Downloads/mm10.ncbiRefSeq.gtf"
mm10_cds = read_gtfcds(input_mm10_gtf)                        

print(len(mm10_cds))
print(mm10_cds[0:5])


472761
[{'chromosome': 'chr1', 'transcriptId': 'XM_006495471.3', 'geneId': 'Lypla1', 'start': 4807734, 'end': 4807982, 'rev': '+'}, {'chromosome': 'chr1', 'transcriptId': 'XM_006495471.3', 'geneId': 'Lypla1', 'start': 4808455, 'end': 4808486, 'rev': '+'}, {'chromosome': 'chr1', 'transcriptId': 'XM_006495471.3', 'geneId': 'Lypla1', 'start': 4828584, 'end': 4828649, 'rev': '+'}, {'chromosome': 'chr1', 'transcriptId': 'XM_006495471.3', 'geneId': 'Lypla1', 'start': 4830268, 'end': 4830315, 'rev': '+'}, {'chromosome': 'chr1', 'transcriptId': 'XM_006495471.3', 'geneId': 'Lypla1', 'start': 4832311, 'end': 4832381, 'rev': '+'}]
CPU times: user 4.38 s, sys: 211 ms, total: 4.6 s
Wall time: 4.66 s


In [69]:
%%time
#sort and merge the mm10_cds for each chromosome

cds_region_chrom = []

for i in mm10_cds:
    if i['chromosome'] == 'chr1':
        cds_region_chrom.append([int(i['start']), int(i['end'])])

print(len(cds_region_chrom))
#print(cds_region_chrom[0:800])
#print(cds_region_chrom[-1])

def cds_merge(intervals):
    intervals_sort = sorted(intervals, key=lambda x: x[0])
    merged = []
    for interval in intervals_sort:
        if not merged or merged[-1][-1] < interval[0]:
            merged.append(interval)
        else:
            merged[-1][-1] = max(merged[-1][-1], interval[-1])
    return merged

result_cds = cds_merge(cds_region_chrom)

print(len(result_cds))
#print(result_cds)


#load/read senEff information from MC38_chromosome_filter_dict
snpEff_chrom = []

for i in MC38_chromosome_filter_dict.keys():
    if i.split(':')[0] == 'chr1':
        snpEff_chrom.append(MC38_chromosome_filter_dict[i])
print(len(snpEff_chrom))
print(snpEff_chrom[-1])


36559
6809
20100
['chr1', '195371622', '.', 'C', 'CAGGGTG', '.', '.', 'AC=1;AN=2;ANN=CAGGGTG|intergenic_region|MODIFIER|Cr2-CHR_END|Cr2-CHR_END|intergenic_region|Cr2-CHR_END|||n.195371622_195371623insAGGGTG||||||;AS_SB_TABLE=0,0|16,15;DP=39;ECNT=1;MBQ=0,37;MFRL=0,260;MMQ=60,60;MPOS=46;POPAF=7.30;RPA=4,5;RU=AGGGTG;SF=0;STR;TLOD=118.29', 'GT:AD:DP:F1R2:F2R1:SB:AF', '0/1:0,31:31:0,17:0,10:0,0,16,15:0.969', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.\n']
CPU times: user 154 ms, sys: 0 ns, total: 154 ms
Wall time: 153 ms


In [52]:
%%time
#use two pointers to compare MC38_chromosome_filter_dict and result_cds_sort

snp, cds = 0, 0
final_result = []

while snp < len(snpEff_chrom) and cds < len(result_cds):
   
    if result_cds[cds][0] <= int(snpEff_chrom[snp][1]) <= result_cds[cds][1]:
        final_result.append(snpEff_chrom[snp])
        snp += 1
        
    elif int(snpEff_chrom[snp][1]) < result_cds[cds][0]:        
        snp += 1
            
    elif int(snpEff_chrom[snp][1]) > result_cds[cds][1]:
        cds += 1
        
print(len(final_result))
#print(final_result)


105
CPU times: user 20.9 ms, sys: 0 ns, total: 20.9 ms
Wall time: 20.4 ms


In [6]:
with open('/home/raymond/Desktop/MC38_summary/MC38_somatic_mutation_calling_filter_snpeff_CDS/two_pointers/MC38_chr1_filter_snpeff_cds_twopointer.csv', 'w', newline="") as f:
    writer = csv.writer(f)
    writer.writerows(final_result)